In [1]:
import import_ipynb
from PUE_internalfunctions import *

importing Jupyter notebook from PUE_internalfunctions.ipynb


# AIRSIDE ECONOMIZER WITH ADIABACTC COOLING

In [2]:
# AIRSIDE ECONOMIZER WITH ADIABACTC COOLING
def PUE_AE_Chiller(w_aech,*data):
    
    T_oa,RH_oa,P_atm,Power_IT=data
    
    UPS_e = w_aech[0]
    PD_lr = w_aech[1]
    L_percentage = w_aech[2]
    
    T_sa_lower = w_aech[3]
    T_sa_upper = w_aech[4]
    dp_sa_lower = w_aech[5]
    dp_sa_upper = w_aech[6]
    RH_sa_lower = w_aech[7]
    RH_sa_upper = w_aech[8]
    
    delta_T_air = w_aech[9]
    delta_T_water = w_aech[10]
    delta_T_CT = w_aech[11]
    
    Fan_Pressure_CRAC = w_aech[12]
    Fan_e_CRAC = w_aech[13]
    Fan_Pressure_CT = w_aech[14]
    Fan_e_CT = w_aech[15]
    
    Pump_Pressure_HD = w_aech[16]
    Pump_e_HD = w_aech[17]
    Pump_Pressure_CW = w_aech[18]
    Pump_e_CW = w_aech[19]
    Pump_Pressure_CT = w_aech[20]
    Pump_e_CT = w_aech[21]
    
    AT_CT = w_aech[22]
    Chiller_load = w_aech[23]
    pCOP = w_aech[24]
    L_to_G = w_aech[25]
    
    #Dewpoint: dp_oa, C
    dp_oa = T_dp(T_oa, RH_oa)
    #Saturation vapor pressure: P_sat, Pa
    P_sat = 611.21 * np.exp(((18.678 - T_oa/234.5) * T_oa)/(T_oa + 257.14))
    #Vapor pressure: P_vapor, Pa
    P_vapor = P_sat * RH_oa/100
    #Absolute humidity: d_oa, g/kg
    d_oa = 622 * (RH_oa/100 * P_sat)/(P_atm - RH_oa/100 * P_sat)
    #Wetbulb temperature of outdoor air: T_wb_oa, C
    T_wb_oa = wet_blub(T_oa,RH_oa,P_atm,P_sat)
    #Density of outdoor air: ad_oa, kg/m^3
    density_oa = air_density(P_atm,T_oa)
    #Enthalpy of outdoor air: E_oa, kJ/kg
    E_oa = 1.01 * T_oa + (2500 + 1.84 * T_oa) * d_oa/1000
    
    #Cooling requirement: KW
    DC_heat_AC = Cooling_Requirement(Power_IT,UPS_e,PD_lr,L_percentage)[4]
    #CRAC supply air drybulb temperature: T_sa, C
    T_sa = T_supplyair(T_sa_lower,T_sa_upper,T_oa)
    #CRAC return air drybulb temperature: T_sa, C
    T_ra = T_supplyair(T_sa_lower,T_sa_upper,T_oa) + delta_T_air
    #Usage scenario of AE: 1/0
    AE_use = Airside_economizer_use_advanced(E_oa,P_atm,T_sa,d_oa,T_oa,dp_sa_lower,dp_sa_upper,RH_sa_lower,RH_sa_upper)[0]
    #Scenario of dehumification: 1/0, typically they don't perform this in Hyperscale DC
    DHD_use = 0 
    #Scenario of dehumification: 1/0
    HD_use = Airside_economizer_use_advanced(E_oa,P_atm,T_sa,d_oa,T_oa,dp_sa_lower,dp_sa_upper,RH_sa_lower,RH_sa_upper)[1]
    #Enthalpy of pre-conditioned outdoor air:kJ/kg
    E_oa_cd = Airside_economizer_use_advanced(E_oa,P_atm,T_sa,d_oa,T_oa,dp_sa_lower,dp_sa_upper,RH_sa_lower,RH_sa_upper)[4]
    #Enthalpy of supply air: kJ/kg
    E_sa = Airside_economizer_use_advanced(E_oa,P_atm,T_sa,d_oa,T_oa,dp_sa_lower,dp_sa_upper,RH_sa_lower,RH_sa_upper)[5]
    #Enthalpy of return air: kJ/kg
    E_ra = E_sa + (1.01)*delta_T_air
    #Massflow of supply air: kJ/kg
    m_sa = DC_heat_AC/(E_ra-E_sa)
    #Massflow of return air: kJ/kg
    if AE_use ==1:
        m_ra = max(0,m_sa*(E_sa-E_oa_cd)/(E_ra-E_oa_cd))
    if AE_use ==0:
        m_ra = m_sa
    #Massflow of outdoor air: kJ/kg
    m_oa_cd = m_sa - m_ra
    m_oa = m_oa_cd/(1+Airside_economizer_use_advanced(E_oa,P_atm,T_sa,d_oa,T_oa,dp_sa_lower,dp_sa_upper,RH_sa_lower,RH_sa_upper)[2]/1000)
    #Heat removed by AE plus adiabatic cooling
    AE_heat_removed =AE_use*m_oa_cd*(E_ra-E_oa_cd)
    #Airside economizer percentage, %
    AE_percentage = AE_heat_removed/DC_heat_AC
    #Density of supply air,kg/m^3
    density_sa = air_density(P_atm,T_sa)
    ###(1.Power)###: CRAC Fan power use: Power_Fan_CRAC, kW
    Power_Fan_CRAC = Fan_Power(m_sa,density_sa,Fan_Pressure_CRAC,Fan_e_CRAC)
    #Humidification amount, kg/s
    hd_amount = m_oa_cd - m_oa
    #Dehumidification amount, kg/s
    dhd_amount = 0
    ###(2.Power)###: Power used by humidification pump, kW
    Power_Pump_hd = Pump_Power(hd_amount,Pump_Pressure_HD,Pump_e_HD,1000)
    #COP of chiller: COP_chiller, kW/kW (regression Mean*(1+%COP))
    COP_chiller = COP_Chiller(Chiller_load,T_wb_oa+AT_CT)*(1+pCOP)
    #Energy used by dehumidification devices, kW
    energy_dhd = 0
    #Additional cooling supplied by chiller: Chiller_heat_removed, kW
    Chiller_heat_removed = DC_heat_AC - AE_heat_removed + energy_dhd
    ###(3.Power)###: Power used by Chiller: Power_Chiller, kW
    Power_Chiller = Chiller_heat_removed/COP_chiller
    #Mass flow rate of chilled water: m_sw = m_rw, kg/s
    m_sw = Chiller_heat_removed/(4.2*(delta_T_water))
    ###(4.Power)###: Power used by chiller pump: Power_Pump_CW, kW
    Power_Pump_CW = Pump_Power(m_sw,Pump_Pressure_CW,Pump_e_CW,1000)
    #Heat removed by cooing tower: CT_heat_removed, kW
    CT_heat_removed = Chiller_heat_removed + Power_Chiller
    #Mass flow rate of cooling tower water: m_CT, kg/s
    m_CT = CT_heat_removed/(4.2*delta_T_CT)
    ###(5.Power)###: Power used by cooling tower water pump: Power_Pump_CT, kW
    Power_Pump_CT = Pump_Power(m_CT,Pump_Pressure_CT,Pump_e_CT,1000)
    ###(6.Power)###: Power used by cooling tower fan: Power_Fan_CT, kW
    Power_Fan_CT = Fan_Power(m_CT/L_to_G,density_oa,Fan_Pressure_CT,Fan_e_CT)
    
    ###(0.Power)###: UPS
    Powerloss_UPS = Cooling_Requirement(Power_IT,UPS_e,PD_lr,L_percentage)[0]
    ###(0.Power)###: PD
    Powerloss_PD = Cooling_Requirement(Power_IT,UPS_e,PD_lr,L_percentage)[1]
    ###(0.Power)###: Lighting
    Power_Lighting = Cooling_Requirement(Power_IT,UPS_e,PD_lr,L_percentage)[2]
    
    # PUE of airside economizer
    PUE_AE_CH = (Power_IT + Powerloss_UPS + Powerloss_PD + Power_Lighting + Power_Fan_CRAC + Power_Pump_hd + Power_Chiller + Power_Pump_CW + Power_Pump_CT + Power_Fan_CT)/Power_IT
    P = [Power_IT,Powerloss_UPS,Powerloss_PD,Power_Lighting,Power_Fan_CRAC,Power_Pump_hd,Power_Chiller,Power_Pump_CW,Power_Pump_CT,Power_Fan_CT]
    return PUE_AE_CH

# WATERSIDE ECONOMIZER USING EVAPORTATIVE COOLING CAPABILITY OF COOLING TOWER

In [4]:
# WATERSIDE ECONOMIZER USING EVAPORTATIVE COOLING CAPABILITY OF COOLING TOWER
def PUE_WEC_Chiller(w_wech,*data):
    
    T_oa,RH_oa,P_atm,Power_IT=data
    
    UPS_e = w_wech[0]
    PD_lr = w_wech[1]
    L_percentage = w_wech[2]
    
    T_sa_setpoint = w_wech[3]
    
    delta_T_air = w_wech[4]
    delta_T_water = w_wech[5]
    delta_T_CT = w_wech[6]
    
    Fan_Pressure_CRAC = w_wech[7]
    Fan_e_CRAC = w_wech[8]
    Fan_Pressure_CT = w_wech[9]
    Fan_e_CT = w_wech[10]
    
    Pump_Pressure_HD = w_wech[11]
    Pump_e_HD = w_wech[12]
    Pump_Pressure_WE = w_wech[13]
    Pump_e_WE = w_wech[14]
    Pump_Pressure_CW = w_wech[15]
    Pump_e_CW = w_wech[16]
    Pump_Pressure_CT = w_wech[17]
    Pump_e_CT = w_wech[18]

    AT_CT = w_wech[19]
    AT_HE = w_wech[20]
    HTE = w_wech[21]
    SHR = w_wech[22]
    Chiller_load = w_wech[23]
    pCOP = w_wech[24]
    L_to_G = w_wech[25]
    
    #Dewpoint: dp_oa, C
    dp_oa = T_dp(T_oa, RH_oa)
    #Saturation vapor pressure: P_sat, Pa
    P_sat = 611.21 * np.exp(((18.678 - T_oa/234.5) * T_oa)/(T_oa + 257.14))
    #Vapor pressure: P_vapor, Pa
    P_vapor = P_sat * RH_oa/100
    #Absolute humidity: d_oa, g/kg
    d_oa = 622 * (RH_oa/100 * P_sat)/(P_atm - RH_oa/100 * P_sat)
    #Wetbulb temperature of outdoor air: T_wb_oa, C
    T_wb_oa = wet_blub(T_oa,RH_oa,P_atm,P_sat)
    #Air density: ad_oa, kg/m^3:
    density_oa = air_density(P_atm,T_oa)
    #Air enthalpy: E_oa, kJ/kg
    E_oa = 1.01 * T_oa + (2500 + 1.84 * T_oa) * d_oa/1000
    
    #Cooling requirements: kW
    DC_heat_AC = Cooling_Requirement(Power_IT,UPS_e,PD_lr,L_percentage)[4]
    #CRAC supply air drybulb temperature: T_sa, C
    T_sa = T_sa_setpoint
    #CRAC return air drybulb temperature: T_sa, C
    T_ra = T_sa + delta_T_air
    #Density of supply air: density_sa, kg/m^3
    density_sa = air_density(P_atm,T_sa)
    # Mass flow rate of supply air: m_sa, kg/s
    m_sa = DC_heat_AC/(1.01 * delta_T_air )
    ###(1.Power)###: CRAC Fan power use: Power_Fan_CRAC, kW
    Power_Fan_CRAC = Fan_Power(m_sa,density_sa,Fan_Pressure_CRAC,Fan_e_CRAC)
    #Latent heat removed by air, DC_heat_latent, kW
    DC_heat_latent = Cooling_Requirement(Power_IT,UPS_e,PD_lr,L_percentage)[6]/SHR - DC_heat_AC
    #Cooling required: Cooling_required, kW
    Cooling_required = Cooling_Requirement(Power_IT,UPS_e,PD_lr,L_percentage)[6] + DC_heat_latent
    #Supply facility water temperature: T_sfw, C
    T_sfw = T_supplywater(T_sa,T_ra,HTE)
    #Return facility water temperature: T_rfw, C
    T_rfw = T_sfw + delta_T_water
    #Estimated temperature of condensate water: T_cw, C
    T_cw = (T_sfw + T_rfw)/2
    #Humidification amount, kg/s
    hd_amount = DC_heat_latent/Latent_heat_vaporization(T_cw)
    ###(2.Power)###: Power used by humidification pump, kW
    Power_Pump_hd = Pump_Power(hd_amount,Pump_Pressure_HD,Pump_e_HD,1000)  
    #Usage scenario of waterside economizer: 1--use, 0--not use
    WE_use = Waterside_economizer_use(T_rfw,T_wb_oa,AT_CT,AT_HE)
    #The heat removed by waterside economizer: WE_heat_removed, kW
    WE_heat_removed = Water_heat_removed(Cooling_required,T_sfw,T_rfw,T_wb_oa,AT_CT,AT_HE)
    #Mass flow rate of facility water: m_sfw = m_rfw, kg/s
    m_sfw = WE_heat_removed/(4.2*delta_T_water)
    ###(3.Power)###: Power used by waterside economizer pump: Power_Pump_WE, kW
    Power_Pump_WE = Pump_Power(m_sfw,Pump_Pressure_WE,Pump_e_WE,1000)
    #Additional cooling needed, if the case, kW
    Chiller_heat_removed = Cooling_required - WE_heat_removed
    #COP of chiller: COP_chiller, kW/kW (regression Mean*(1+%COP))
    COP_chiller = COP_Chiller(Chiller_load,T_wb_oa+AT_CT)*(1+pCOP)
    ###(4.Power)###: Power used by Chiller: Power_Chiller, kW
    Power_Chiller = Chiller_heat_removed/COP_chiller
    #Mass flow rate of chiller water: m_sw = m_rw, kg/s
    m_sw = Chiller_heat_removed/(4.2*(delta_T_water))
    ###(5.Power)###: Power used by chilled water pump: Power_Pump_CW, kW
    Power_Pump_CW = Pump_Power(m_sw,Pump_Pressure_CW,Pump_e_CW,1000)
    #Heat removed by cooing tower: CT_heat_removed, kW
    CT_heat_removed = Cooling_required + Power_Chiller
    #Mass flow rate of cooling tower water: m_CT, kg/s
    m_CT = CT_heat_removed/(4.2*delta_T_CT)
    ###(6.Power)###: Power used by cooling tower water pump: Power_Pump_CT, kW
    Power_Pump_CT = Pump_Power(m_CT,Pump_Pressure_CT,Pump_e_CT,1000)
    ###(7.Power)###: Power used by cooling tower fan: Power_Fan_CT, kW
    Power_Fan_CT = Fan_Power(m_CT/L_to_G,density_oa,Fan_Pressure_CT,Fan_e_CT)
    
    ###(0.Power)###: UPS
    Powerloss_UPS = Cooling_Requirement(Power_IT,UPS_e,PD_lr,L_percentage)[0]
    ###(0.Power)###: PD
    Powerloss_PD = Cooling_Requirement(Power_IT,UPS_e,PD_lr,L_percentage)[1]
    ###(0.Power)###: Lighting
    Power_Lighting = Cooling_Requirement(Power_IT,UPS_e,PD_lr,L_percentage)[2]
    
    PUE_WE_CH = (Power_IT + Powerloss_UPS + Powerloss_PD + Power_Lighting + Power_Fan_CRAC + Power_Pump_hd + Power_Pump_WE + Power_Chiller + Power_Pump_CW + Power_Pump_CT + Power_Fan_CT)/Power_IT
    
    P = [Power_IT,Powerloss_UPS,Powerloss_PD,Power_Lighting,Power_Fan_CRAC,Power_Pump_hd,Power_Pump_WE,Power_Chiller,Power_Pump_CW ,Power_Pump_CT ,Power_Fan_CT]
    return PUE_WE_CH

# WATERSIDE ECONOMIZER USING SEAWATER

In [5]:
# WATERSIDE ECONOMIZER USING SEAWATER
def PUE_WES_Chiller(w_wech,*data):
       
    T_sea,P_atm,Power_IT=data
    
    UPS_e = w_wech[0] 
    PD_lr = w_wech[1]
    L_percentage = w_wech[2]
    
    T_sa_setpoint = w_wech[3]
    
    delta_T_air = w_wech[4]
    delta_T_water = w_wech[5]
    delta_T_sea = w_wech[6]
    
    Fan_Pressure_CRAC = w_wech[7]
    Fan_e_CRAC = w_wech[8]
    
    Pump_Pressure_HD = w_wech[9]
    Pump_e_HD = w_wech[10]
    Pump_Pressure_WE = w_wech[11]
    Pump_e_WE = w_wech[12]
    Pump_Pressure_CW = w_wech[13]
    Pump_e_CW = w_wech[14]
    Pump_Pressure_sea = w_wech[15]
    Pump_e_sea = w_wech[16]
    
    AT_HE = w_wech[17]
    HTE = w_wech[18]
    SHR = w_wech[19]
    Chiller_load = w_wech[20]
    pCOP = w_wech[21]
    
    #Cooling requirements: kW
    DC_heat_AC = Cooling_Requirement(Power_IT,UPS_e,PD_lr,L_percentage)[4]
    #CRAC supply air drybulb temperature: T_sa, C
    T_sa = T_sa_setpoint
    #CRAC return air drybulb temperature: T_sa, C
    T_ra = T_sa + delta_T_air
    #Density of supply air: density_sa, kg/m^3
    density_sa = air_density(P_atm,T_sa)
    #Mass flow rate of supply air: m_sa, kg/s
    m_sa = DC_heat_AC/(1.01 * delta_T_air )
    ###(1.Power)###: CRAC Fan power use: Power_Fan_CRAC, kW
    Power_Fan_CRAC = Fan_Power(m_sa,density_sa,Fan_Pressure_CRAC,Fan_e_CRAC)
    #Latent heat removed by air, DC_heat_latent, kW
    DC_heat_latent = Cooling_Requirement(Power_IT,UPS_e,PD_lr,L_percentage)[6]/SHR - DC_heat_AC
    #Cooling required: Cooling_required, kW
    Cooling_required = Cooling_Requirement(Power_IT,UPS_e,PD_lr,L_percentage)[6] + DC_heat_latent
    #Supply facility water temperature: T_sfw, C
    T_sfw = T_supplywater(T_sa,T_ra,HTE)
    #Return facility water temperature: T_rfw, C
    T_rfw = T_sfw + delta_T_water
    #Estimated temperature of condensate water: T_cw, C
    T_cw = (T_sfw + T_rfw)/2
    #Humidification amount, kg/s
    hd_amount = DC_heat_latent/Latent_heat_vaporization(T_cw)
    ###(2.Power)###: Power used by humidification pump, kW
    Power_Pump_hd = Pump_Power(hd_amount,Pump_Pressure_HD,Pump_e_HD,1000)
    #Usage scenario of waterside economizer: 1--use, 0--not use
    WE_use = Waterside_economizer_use(T_rfw,T_sea,0,AT_HE)
    #The heat removed by waterside economizer: WE_heat_removed, kW
    WE_heat_removed = Water_heat_removed(Cooling_required,T_sfw,T_rfw,T_sea,0,AT_HE)
    #Mass flow rate of facility water: m_sfw = m_rfw, kg/s
    m_sfw = WE_heat_removed/(4.2*delta_T_water)
    ###(3.Power)###: Power used by waterside economizer pump: Power_Pump_WE, kW
    Power_Pump_WE = Pump_Power(m_sfw,Pump_Pressure_WE,Pump_e_WE,1000)
    #Additional cooling needed, if the case, kW
    Chiller_heat_removed = Cooling_required - WE_heat_removed
    #COP of chiller: COP_chiller, kW/kW (regression Mean*(1+%COP))
    COP_chiller = COP_Chiller(Chiller_load,T_sea+0)*(1+pCOP)
    ###(4.Power)###: Power used by Chiller: Power_Chiller, kW
    Power_Chiller = Chiller_heat_removed/COP_chiller
    #Mass flow rate of chiller water: m_sw = m_rw, kg/s
    m_sw = Chiller_heat_removed/(4.2*(delta_T_water))
    ###(5.Power)###: Power used by chilled water pump: Power_Pump_CW, kW
    Power_Pump_CW = Pump_Power(m_sw,Pump_Pressure_CW,Pump_e_CW,1000)
    #Heat removed by sea water: CT_heat_removed, kW
    SEA_heat_removed = Cooling_required + Power_Chiller
    # Mass flow rate of sea water: m_CT, kg/s
    m_sea = SEA_heat_removed/(4.2*delta_T_sea) 
    ###(6.Power)###: Power used by seawater pump
    Power_Pump_sea = Pump_Power(m_sea,Pump_Pressure_sea,Pump_e_sea,1000)
    
    ###(0.Power)###: UPS
    Powerloss_UPS = Cooling_Requirement(Power_IT,UPS_e,PD_lr,L_percentage)[0]
    ###(0.Power)###: PD
    Powerloss_PD = Cooling_Requirement(Power_IT,UPS_e,PD_lr,L_percentage)[1]
    ###(0.Power)###: Lighting
    Power_Lighting = Cooling_Requirement(Power_IT,UPS_e,PD_lr,L_percentage)[2]
    
    PUE_WE_CH = (Power_IT + Powerloss_UPS + Powerloss_PD + Power_Lighting + Power_Fan_CRAC + Power_Pump_hd + Power_Pump_WE + Power_Chiller + Power_Pump_CW + Power_Pump_sea)/Power_IT
    P = [Power_IT, Powerloss_UPS,Powerloss_PD,Power_Lighting, Power_Fan_CRAC,Power_Pump_hd, Power_Pump_WE, Power_Chiller,Power_Pump_CW ,Power_Pump_sea]
    return PUE_WE_CH